### Scraping official gazetta articles of given links in the csv file

In [1]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
import re
import string
import unidecode


sys.setrecursionlimit(1000000)
data=pd.read_csv('./data/KPMG _Tax_Case_Data _Set.csv')

In [5]:
data.head(3)

,Date,Title,Numac,Link FR,Link NL
0,14/01/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...
1,16/01/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...
2,16/01/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...


In [3]:
data.drop_duplicates(inplace=True)

In [ ]:
#Url is in framesets we need the second frame
data['Link FR'] = data['Link FR'].str.replace('article.pl','article_body.pl')
data['Link NL'] = data['Link NL'].str.replace('article.pl','article_body.pl')

In [ ]:
#Cleaning the scraped text

def clean(_a:str):
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', _a)
    
    document_test= unidecode.unidecode(d)
    document_test = document_test.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', document_test)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    document_test = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    
 
    document_test = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', document_test) 
   
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    return document_test

In [ ]:
#Scrape the article 
links=list(data['Link NL'])  #first fremch than dutch 'Link NL'
#links=list(data['Link FR'])

nl_list=[]
_count=0 # to check which line 
for a in links:
    _count+=1
    res = requests.get(a)
    soup = BeautifulSoup(res.text, 'html.parser')
    for sup in soup.find_all('sup'):
        sup.unwrap()
   
     
    text=soup.text
    text=text.replace('\n',"")
    lst=text.split('Numac :')[1].split(text.split('Numac :')[2])
  

    article=lst[1].split('begin eerste woord laatste')[0].strip()
    article=clean(article)
    nl_list.append(article)
    print(_count)
    

In [ ]:
data['nl_body'] = nl_list
#data['fr_body'] = nl_list


In [ ]:
data.head(5)

In [ ]:
data.to_csv("data_01.csv",sep="|",index=False,encoding='utf-8')

In [ ]:
#test saved csv file
df=pd.read_csv("data_01.csv",sep="|",encoding='utf-8')